In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
from torch.utils.data import Dataset,DataLoader
from torchvision import models
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import glob
import cv2
import albumentations
from tqdm import tqdm
import copy
from PIL import Image
from torch.optim import lr_scheduler

In [ ]:
training_img = os.listdir("../input/training")
test_img = os.listdir("../input/test")
print(len(training_img))
print(len(test_img))

In [ ]:
train_df = pd.read_csv("../input/training_frames_keypoints.csv")
test_df = pd.read_csv("../input/test_frames_keypoints.csv")

In [ ]:
def show_keypoints(n):
  image_name = train_df.iloc[n,0]
  key_pts = np.array(train_df.iloc[n, 1:])
  key_pts = key_pts.astype('float').reshape(-1, 2)
  image = os.path.join("../input/training",image_name)
  image = mpimg.imread(image)
  print(np.array(image).shape)
  plt.imshow(image)
  plt.scatter(key_pts[:,0],key_pts[:,1],c='m',s=20,marker=".")
  plt.show()
show_keypoints(3)
show_keypoints(4)
show_keypoints(5)
show_keypoints(6)

In [ ]:
class FaceKeyPts(Dataset):
  def __init__(self,dataframe,root_dir,transform = None,new_dim = None):
    self.dataframe = dataframe
    self.root_dir = root_dir
    self.transform = transform
    self.new_dim = new_dim
  def __len__(self):
    return self.dataframe.shape[0]
  def __getitem__(self,idx):
    image_name = self.dataframe.iloc[idx,0]
    image = os.path.join(self.root_dir,image_name)
    image = Image.open(image)
    if self.new_dim is not None:
      image = image.resize((self.new_dim))
    image = np.array(image)
    if image.shape[2] == 4:
      image = image[:,:,0:3]
    image = np.array(image)
    if self.transform is not None:
      image = self.transform(image = np.array(image))["image"]
    image = np.transpose(image, (2, 0, 1)).astype(np.float32)
    key_pts = np.array(self.dataframe.iloc[idx,1:]).reshape((-1,2)).astype(float)

    sample = {
        "image" : torch.tensor(image , dtype = torch.float),
        "key_pts" : torch.tensor(key_pts , dtype = torch.long)
    }

    return sample

In [ ]:
TRAIN_BATCH_SIZE = 16
VAL_BATCH_SIZE = 8
EPOCHS = 50
MODEL = "vgg"
NUM_CLASSES = 136
DEVICE ="cuda"

In [ ]:
def set_parameters_grad(model,feature_extract = False):
    if feature_extract:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
def initialize_model(model,num_class,feature_extract=True,pretrained=True):
    
    model_ft = None
    input_size = 0
    
    if model == "resnet":
        model_ft = models.resnet50(pretrained = pretrained)
        set_parameters_grad(model_ft,feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_class)

        input_size = (224,224)
    
    elif model == "vgg":
        model_ft = models.vgg11_bn(pretrained=pretrained)
        set_parameters_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_class)
        input_size = (224,224)
    
    elif model == "alexnet":
        model_ft = models.alexnet(pretrained=pretrained)
        set_parameters_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.fc = nn.Linear(num_ftrs, num_class)

        input_size = (224,224)
    
    elif model == "densenet":
        self.model = models.densenet121(pretrained = pretrained)
        set_parameters_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_class)
        input_size = (224,224)
    
    elif model == "squeezenet":
        model_ft = models.squeezenet1_0(pretrained=pretrained)
        set_parameters_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_class, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_class
        input_size = (224,224)
    
    else:
        print("invalid model name")
    
    return model_ft,input_size

In [ ]:

def train_model(model,dataloader,criterion,optimizer,scheduler = None,epochs=5):
    best_model_weights = copy.deepcopy(model.state_dict())
    train_losses = []
    val_losses = []
    for epoch in tqdm(range(epochs)):
        lowest_val_loss = 1000000.0
        train_loss = 0.0
        val_loss = 0.0
        print(f"EPOCH:{epoch}/{epochs-1}")
        print("-"*20)
        for phase in ["train","val"]:
            num = 0
            if phase == "train":
                model.train()
                bs = TRAIN_BATCH_SIZE
            else:
                model.eval()
                bs = VAL_BATCH_SIZE
        
            running_loss = 0.0
            running_corrects = 0

            for i,batch in enumerate(dataloader[phase]):
                inputs = batch["image"].to('cuda')
                labels = batch["key_pts"].to('cuda')
 
                
                labels = labels.view(labels.size(0), -1)
                labels = labels.type(torch.cuda.FloatTensor)
                inputs = inputs.type(torch.cuda.FloatTensor)
                if phase == "train":
                    optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    loss = criterion(outputs,labels)
                print(f"phase : {phase} , loss : {loss}")
                if phase == "train":
                    loss.backward()
                    optimizer.step()
                if phase == "train":
                  train_loss += loss.item()*inputs.shape[0]
                if phase == "val":
                  val_loss += loss.item()*inputs.shape[0]
        
            
        
        scheduler.step()
        train_loss = np.sqrt(train_loss/3462)
        val_loss = np.sqrt(val_loss/2308)

        if val_loss < lowest_val_loss:
            lowest_val_loss = val_loss
            best_weights = copy.deepcopy(model.state_dict())
        print(f"epoch no. : {epoch} , train_loss : {train_loss} , val_loss : {val_loss}")
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
    model.load_state_dict(best_weights)
    print(lowest_val_loss)
    return model

In [ ]:
def run():
  train_root = "../input/training"
  val_root = "../input/test"

  train_df = pd.read_csv("../input/training_frames_keypoints.csv")
  val_df = pd.read_csv("../input/test_frames_keypoints.csv")
  model_ft, input_size = initialize_model(model = MODEL,num_class = NUM_CLASSES,feature_extract = False,pretrained=True)
  model_ft.to("cuda")
  train_aug = albumentations.Compose([
      albumentations.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225]),
  ])

  val_aug = albumentations.Compose([
      albumentations.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])                                                                  
  ])
                                


  train_dataset = FaceKeyPts(
      dataframe = train_df,
      root_dir = train_root,
      transform = train_aug,
      new_dim = input_size
      )
  val_dataset = FaceKeyPts(
      dataframe = val_df,
      root_dir = val_root,
      transform = val_aug,
      new_dim = input_size
  )

  train_dataloader = DataLoader(
      train_dataset,
      batch_size = TRAIN_BATCH_SIZE,
      shuffle = True,
      num_workers = 0
  )

  val_dataloader = DataLoader(
      val_dataset,
      batch_size = VAL_BATCH_SIZE,
      shuffle = False,
      num_workers = 8
  )

  dataloader = {
        "train":train_dataloader,
        "val":val_dataloader
  }

  
  optimizer = torch.optim.Adam(model_ft.parameters(),lr = 0.005)
  exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.05)
  lossfn = nn.MSELoss()
  model = train_model(model_ft,dataloader,criterion = lossfn,optimizer=optimizer,scheduler = exp_lr_scheduler,epochs=30)
  model_path = "model.bin"
  torch.save(model.state_dict(), model_path)


In [ ]:
 run()